## Finding Path 1
## 1. Set configurations

In [ ]:
%reset

In [1]:
from sqlalchemy import create_engine, MetaData, Table
import pandas as pd
from tqdm import tqdm
from dataclasses import dataclass
from typing import List
%cd ~/hillhouse
from utils import secrets

# This option allows dataframes to be displayed on one line
pd.set_option("display.width", 250)
# Centers column headers in output
pd.set_option('colheader_justify', 'center')
# Narrows width of individual columns, so all columns can be seen in output
pd.set_option('display.max_colwidth', 40)

sqlEngine = create_engine(f'mysql+pymysql://{secrets.sqlUser}:{secrets.sqlPass}@{secrets.sqlHost}/defiData', pool_recycle=3600)
dbConnection = sqlEngine.connect()
meta = MetaData(bind=dbConnection)
algoResults = Table('algoResults2', meta, autoload_with=sqlEngine)
addrGroups = Table('addrGroups2', meta, autoload_with=sqlEngine)

/home/mdd39/hillhouse


## 2. Find eligible addresses
### 2a. Select addresses from each protocol

In [2]:
%%time
# Dataframes from SQL
# For maker and aave queries, have to use Filter view, because some rows in the oroginal table are not related to the scope of our analysis
maker_vaults = pd.read_sql('SELECT vaultID, vaultOwnerAddr AS addr1, proxyOwnerAddr as addr2, urnHandlerAddr as addr3 FROM makerVaults', dbConnection)
maker = pd.read_sql('SELECT addr1 FROM makerV2Filter GROUP BY addr1', dbConnection)
aave = pd.read_sql('SELECT addr1 FROM aaveV1Filter GROUP BY addr1', dbConnection)
compound = pd.read_sql('SELECT usrAddr FROM compoundV2 GROUP BY usrAddr', dbConnection)
uniswap_send = pd.read_sql('SELECT sendAddr FROM uniswapV2 uv GROUP BY sendAddr;', dbConnection)
uniswap_receive = pd.read_sql('SELECT receiveAddr FROM uniswapV2 uv GROUP BY receiveAddr;', dbConnection)

# Sets from SQL dataframes
compound_set = set(compound.usrAddr)
aave_set = set(aave.addr1)
uniswap_set = set(uniswap_send.sendAddr) | set(uniswap_receive.receiveAddr)
maker_set = set(maker.addr1)

# Combinations/lists of sets
all_addr = aave_set | compound_set | maker_set | uniswap_set
protocol_sets = [compound_set, uniswap_set, maker_set, aave_set]

CPU times: user 6.04 s, sys: 718 ms, total: 6.76 s
Wall time: 16.3 s


### 2b. Group addresses via Maker Vaults

In [3]:
%%time
max_vault = max(maker_vaults['vaultID'])

addr_group = []
addr_ungroup = set()

# Create list of sets, only on address columns
temp = maker_vaults.iloc[:,1:4].apply(set, axis=1)

for x in tqdm(range(1,max_vault)):
    # Get current row set, eliminate None
    cur_row_set = temp[x]
    cur_row_set.discard(None)

    # If first element, then just add to the list of address groups
    if x == 1:
        addr_group.append(cur_row_set)
    
    else:
        # If current address group has some address in common with previous addresses
        if bool(addr_ungroup & cur_row_set):
            # Look through list, add addresses to list element (set) that matches
            for y in range(len(addr_group)):
                existing_addr_set = addr_group[y]
                if bool(existing_addr_set & cur_row_set):
                    addr_group[y] |= cur_row_set
                    break
        # If nothing in common, then just append new group to the address list
        else:
            addr_group.append(cur_row_set)

    # Intersection current row with addr_ungroup (do whether x == 1 or not). Must be done after all checks performed
    addr_ungroup |= cur_row_set


100%|██████████| 20842/20842 [00:16<00:00, 1260.54it/s]CPU times: user 16.8 s, sys: 215 ms, total: 17.1 s
Wall time: 16.9 s



### 2c. Add addresses from transactions with value (Aave, Compound, Maker, Uniswap)

In [4]:
%%time
# Add addresses from all relevant transactions (Aave/Comp/Maker/Uniswap - only borrow/lend)
for addr in tqdm(all_addr):
    # If addresses in common
    if (addr in addr_ungroup):
        # Look through list, add addresses to list element (set) that matches
        for y in range(len(addr_group)):
            existing_addr_set = addr_group[y]
            if addr in existing_addr_set:
                addr_group[y].add(addr)
                break
    # If nothing in common, then just append new group to the address list (as a set)
    else:
        addr_group.append({addr})
    
    addr_ungroup.add(addr)

print(len(addr_group))
print(len(addr_ungroup))


100%|██████████| 506659/506659 [00:31<00:00, 15836.63it/s]503473
538061
CPU times: user 31.9 s, sys: 492 ms, total: 32.4 s
Wall time: 32 s



### 2d. Create list of eligible address groups (appearing in at least two protocols)

In [5]:
addr_group_eligible = []
addr_ungroup_eligible = set()

for group in addr_group:
    total_protocols_in = 0

    for i in range(4):
        protocol_set = protocol_sets[i]
        in_protocol = int(bool(group & protocol_set))
        total_protocols_in += in_protocol
        if total_protocols_in > 1:
            addr_group_eligible.append(group)
            addr_ungroup_eligible |= group
            break

print(f'Groups eligible: {round(len(addr_group_eligible) / len(addr_group) * 100,2)}% ({len(addr_group_eligible)} out of {len(addr_group)})')

print(f'Addresses eligible: {round(len(addr_ungroup_eligible) / len(addr_ungroup) * 100,2)}% ({len(addr_ungroup_eligible)} out of {len(addr_ungroup)})')

Groups eligible: 5.79% (29149 out of 503473)
Addresses eligible: 8.78% (47216 out of 538061)


### 2e. Add address group to SQL

In [6]:
num_groups = len(addr_group_eligible)
sqlArr = []
for x in tqdm(range(num_groups)):
    group = addr_group_eligible[x]
    for addr in group:
        sqlArr.append({
                'groupID': x, 
                'address': addr})

ins = addrGroups.insert()
dbConnection.execute(ins, sqlArr)

100%|██████████| 29149/29149 [00:00<00:00, 198704.93it/s]


## 3. Analyze transactions by address group
### 3a. Select eligible transactions

In [11]:
%%time
sql_list = str(tuple([key for key in addr_ungroup_eligible])).replace(',)', ')')
query ="SELECT * FROM mergeRecordsCache WHERE addr1 IN {sql_list}".format(sql_list=sql_list)
eligible_transac = pd.read_sql(query, dbConnection)
eligible_transac.sort_values(by='blockNumber', inplace=True)
eligible_transac.head()

CPU times: user 1min 10s, sys: 4.86 s, total: 1min 15s
Wall time: 2min 34s


,id,blockTime,blockNumber,trxHash,addr1,addr2,token1Amt,token1Symbol,token1Usd,token2Amt,token2Symbol,token2Usd,protocol,trxType,vaultID,newToken2Usd
128496,5280470,2020-05-04 13:22:21,10000002,0x621cc227d668e00d6e46fa64a38fd0e167...,0x1ffc20e8af1ff9bb6c6ccf01e0725e867e...,0xac4fd02c99acb23f6eee2ac502e12bd84a...,0.10,WBTC,872.450000,0.000000,DAI,0.000000,Maker,frob,8822.0,None
128503,5280500,2020-05-04 13:24:00,10000008,0x6051b41aef03fcfacc80901e1f6456d947...,0x1ffc20e8af1ff9bb6c6ccf01e0725e867e...,0xac4fd02c99acb23f6eee2ac502e12bd84a...,0.00,WBTC,0.000000,433.000000,DAI,436.028835,Maker,frob,8822.0,None
99296,291894,2020-05-04 13:24:12,10000009,0x33c1273528c4d851dca4d71a31ea191e3c...,0x30fd7ee6cc6f9739b51d47f6a95a26aca2...,None,333.16,DAI,335.490454,16286.435004,None,NaN,Compound,Mint,NaN,None
663206,5257949,2020-05-04 13:26:20,10000013,0x46fedc1abffe6db6dee003a71004c1b084...,0x1e89cbb9a1e368263e23dde354c3d4fab6...,0x967bafe98dee2d899780e7125b067741d5...,0.00,WETH,0.000000,1499.986822,DAI,1510.479230,Maker,frob,6918.0,None
951375,5280366,2020-05-04 13:27:13,10000015,0xd6dfa658447dbfd58f8cdc3551eda59ee2...,0xddf3a1bc55ddcc5ca57c06a57f51e5fc04...,0x1d18a0fdff04c4c462b42bccac22c1df1f...,0.00,WETH,0.000000,-9861.787498,DAI,-9930.770701,Maker,frob,8758.0,None


### 3b. Analyze transaction for each group, add to SQL

In [10]:
%%time

@dataclass
class Currency:
    wallet_free = 0
    wallet_debt = 0

    maker_withdraw_debt = 0 # amount of debt withdrawn, always negative (same for other protocols)
    maker_debt_collat = 0
    maker_free_collat = 0
    sum_maker_debt_lock = 0
    sum_maker_free_lock = 0
    sum_maker_debt_lock_usd = 0
    sum_maker_free_lock_usd = 0

    compound_withdraw_debt = 0 
    compound_debt_collat = 0
    compound_free_collat = 0
    sum_compound_debt_lock = 0
    sum_compound_free_lock = 0
    sum_compound_debt_lock_usd = 0
    sum_compound_free_lock_usd = 0

    aave_withdraw_debt = 0
    aave_debt_collat = 0
    aave_free_collat = 0
    sum_aave_debt_lock = 0
    sum_aave_free_lock = 0
    sum_aave_debt_lock_usd = 0
    sum_aave_free_lock_usd = 0

sum_tokens = {}
sqlArr = []
step = 100

# Types
# 1a: Transaction-level print-out (print statistics for every transaction)
# 1b: Currency-level print-out (print summary statistics across multiple address groups, by currency and protocol)
# 1c: Total (print summary statistics, by total USD without further break-out)
# 2: SQL insert
phase = '2'

query ="select max(groupID) from algoResults2"
results = pd.read_sql(query, dbConnection)
if results.iat[0,0] is None:
    min_id = 0
else:
    min_id = results.iat[0,0]  # Get next group ID (hasn't been entered yet)
max_id = len(addr_group_eligible)
i = 0

for x in tqdm(range(min_id, max_id)):
    i += 1
    pass
    group = addr_group_eligible[x]
    group_tokens = {}
    group_transac = eligible_transac.loc[((eligible_transac['addr1'].isin(group))) | ((eligible_transac['addr2'].isin(group)))]

    # NOTE: zip + to_dict(list) is the fastest method for iteration, per SO discussion
    simple_list = zip(*group_transac.to_dict("list").values())
    for transac in simple_list:
        trxId = transac[0]
        blockTime = transac[1]
        token1Amt = transac[6]
        token1Symbol = transac[7]
        token1Usd = transac[8]
        token2Amt = transac[9]
        token2Symbol = transac[10]
        token2Usd = transac[11]
        protocol = transac[12]
        trxType = transac[13]

        lockTransac = False

        # Create template for token, if none
        if token1Symbol is not None:
            if token1Symbol not in group_tokens.keys():
                group_tokens[token1Symbol] = Currency()
            if token1Symbol not in sum_tokens.keys():
                sum_tokens[token1Symbol] = Currency()
        if token2Symbol is not None:
            if token2Symbol not in group_tokens.keys():
                group_tokens[token2Symbol] = Currency()
            if token2Symbol not in sum_tokens.keys():
                sum_tokens[token2Symbol] = Currency()

        # Uniswap, Maker
        if protocol in ('Uniswap', 'Maker'):
            # Uniswap
            if trxType == 'Swap':
                # Amounts are from perspective of sender. Therefore, reverse balances to get perspective of receiver
                # Start with the amount getting sent (originally positive), because that determines whether any debt is exchanged
                if token1Amt >= token2Amt:
                    sentAmt = token1Amt
                    sentTokenSymbol = token1Symbol

                    receivedAmt = token2Amt
                    receivedtokenSymbol = token2Symbol

                    action1 = 'sent to'
                    action2 = 'received from'
                else:
                    sentAmt = token2Amt
                    sentTokenSymbol = token2Symbol

                    receivedAmt = token1Amt
                    receivedtokenSymbol = token1Symbol

                    action1 = 'received from'
                    action2 = 'sent to'
                
                sentDebtAmt = min(group_tokens[sentTokenSymbol].wallet_debt, sentAmt)
                sentFreeAmt = max(0, sentAmt - sentDebtAmt)

                if sentAmt == 0:
                    debtPct = 0
                else:
                    debtPct = sentDebtAmt / sentAmt
                receivedDebtAmt = debtPct * receivedAmt
                receivedFreeAmt = (1 - debtPct) * receivedAmt

                group_tokens[sentTokenSymbol].wallet_debt = max(0, group_tokens[sentTokenSymbol].wallet_debt - sentDebtAmt)
                group_tokens[sentTokenSymbol].wallet_free = max(0, group_tokens[sentTokenSymbol].wallet_free - sentFreeAmt)

                # receivedAmt is originally negative (from perspective of sender), so subtracting receivedDebAmt and receivedFreeAmt
                # is actually adding those balaces
                group_tokens[receivedtokenSymbol].wallet_debt = max(0, group_tokens[receivedtokenSymbol].wallet_debt \
                    - receivedDebtAmt)
                group_tokens[receivedtokenSymbol].wallet_free = max(0, group_tokens[receivedtokenSymbol].wallet_free \
                    - receivedFreeAmt)
            
            # Maker
            elif trxType == 'frob':
                # Subtract token1 (dink = collateral), add token2 (dart = debt) to wallet balance
                # Reverse amounts for Maker balance
                
                if token1Amt > 0:
                    lockTransac = True
                    debtAmt = min(group_tokens[token1Symbol].wallet_debt, token1Amt)
                    freeAmt = max(0, token1Amt - debtAmt)
                    if token1Amt == 0:
                        debtPct = 0
                    else:
                        debtPct = debtAmt / token1Amt
                    debtAmtUsd = token1Usd * debtPct
                    freeAmtUsd = token1Usd * (1 - debtPct)

                    group_tokens[token1Symbol].wallet_debt = max(0, group_tokens[token1Symbol].wallet_debt - debtAmt)
                    group_tokens[token1Symbol].wallet_free = max(0, group_tokens[token1Symbol].wallet_free - freeAmt)
                    group_tokens[token1Symbol].maker_debt_collat += debtAmt
                    group_tokens[token1Symbol].maker_free_collat += freeAmt

                    sum_tokens[token1Symbol].sum_maker_debt_lock += debtAmt
                    sum_tokens[token1Symbol].sum_maker_free_lock += freeAmt
                    sum_tokens[token1Symbol].sum_maker_debt_lock_usd += debtAmtUsd
                    sum_tokens[token1Symbol].sum_maker_free_lock_usd += freeAmtUsd

                    action1 = 'locked in'
                else:
                    debtAmt = min(group_tokens[token1Symbol].maker_debt_collat, -token1Amt)
                    freeAmt = max(0, -token1Amt - debtAmt)

                    group_tokens[token1Symbol].maker_debt_collat = max(0, group_tokens[token1Symbol].maker_debt_collat - debtAmt)
                    group_tokens[token1Symbol].maker_free_collat = max(0, group_tokens[token1Symbol].maker_free_collat - freeAmt)
                    group_tokens[token1Symbol].wallet_debt += debtAmt
                    group_tokens[token1Symbol].wallet_free += freeAmt

                    action1 = 'unlocked from'

                if token2Amt > 0:
                    action2 = 'withdrawn from'
                else:
                    action2 = 'repaid to'

                # Same debt calculations for withdraw/repay
                group_tokens[token2Symbol].wallet_debt = max(0, group_tokens[token2Symbol].wallet_debt + token2Amt)    
                group_tokens[token2Symbol].maker_withdraw_debt = min(0, group_tokens[token2Symbol].maker_withdraw_debt - token2Amt)
                
            # Same message for both Uniswap and Maker
            msg_params = {
                'token1Amt': abs(round(token1Amt,2)),
                'token1Symbol': token1Symbol,
                'action1': action1,
                'token2Amt': abs(round(token2Amt,2)),
                'token2Symbol': token2Symbol,
                'action2': action2,
                'funds_location': protocol,
            }

            msg_text = """\n{token1Amt} {token1Symbol} {action1}, {token2Amt} {token2Symbol} {action2} {funds_location}. """.format(**msg_params)

        # Compound, Aave
        elif protocol in ('Compound', 'Aave'):
            if trxType in ('Mint', 'Deposit'):
                lockTransac = True

                debtAmt = min(group_tokens[token1Symbol].wallet_debt, token1Amt)
                freeAmt = max(0, token1Amt - debtAmt)
                if token1Amt == 0:
                    debtPct = 0
                else:
                    debtPct = debtAmt / token1Amt
                debtAmtUsd = token1Usd * debtPct
                freeAmtUsd = token1Usd * (1 - debtPct)

                group_tokens[token1Symbol].wallet_debt = max(0, group_tokens[token1Symbol].wallet_debt - debtAmt)
                group_tokens[token1Symbol].wallet_free = max(0, group_tokens[token1Symbol].wallet_free - freeAmt)
                
                if protocol == 'Compound':
                    group_tokens[token1Symbol].compound_debt_collat += debtAmt
                    group_tokens[token1Symbol].compound_free_collat += freeAmt

                    sum_tokens[token1Symbol].sum_compound_debt_lock += debtAmt
                    sum_tokens[token1Symbol].sum_compound_free_lock += freeAmt
                    sum_tokens[token1Symbol].sum_compound_debt_lock_usd += token1Usd * debtPct
                    sum_tokens[token1Symbol].sum_compound_free_lock_usd += token1Usd * (1 - debtPct)
                else:
                    group_tokens[token1Symbol].aave_debt_collat += debtAmt
                    group_tokens[token1Symbol].aave_free_collat += freeAmt

                    sum_tokens[token1Symbol].sum_aave_debt_lock += debtAmt
                    sum_tokens[token1Symbol].sum_aave_free_lock += freeAmt
                    sum_tokens[token1Symbol].sum_aave_debt_lock_usd += debtAmtUsd
                    sum_tokens[token1Symbol].sum_aave_free_lock_usd += freeAmtUsd

                action = 'locked in'
            if trxType in ('Redeem', 'RedeemUnderlying'):
                if protocol == 'Compound':
                    debtAmt = min(group_tokens[token1Symbol].compound_debt_collat, token1Amt)
                    freeAmt = max(0, token1Amt - debtAmt)

                    group_tokens[token1Symbol].compound_debt_collat = max(0, group_tokens[token1Symbol].compound_debt_collat \
                        - debtAmt)
                    group_tokens[token1Symbol].compound_free_collat = max(0, group_tokens[token1Symbol].compound_free_collat \
                        - freeAmt)
                else:
                    debtAmt = min(group_tokens[token1Symbol].aave_debt_collat, token1Amt)
                    freeAmt = max(0, token1Amt - debtAmt)

                    group_tokens[token1Symbol].aave_debt_collat = max(0, group_tokens[token1Symbol].aave_debt_collat - debtAmt)
                    group_tokens[token1Symbol].aave_free_collat = max(0, group_tokens[token1Symbol].aave_free_collat - freeAmt)

                group_tokens[token1Symbol].wallet_debt += debtAmt
                group_tokens[token1Symbol].wallet_free += freeAmt
                
                action = 'unlocked from'
            if trxType == 'Borrow': # Borrow same transaction for both Aave and Compound
                if protocol == 'Compound':
                    group_tokens[token1Symbol].compound_withdraw_debt = min(0, group_tokens[token1Symbol].compound_withdraw_debt \
                         - token1Amt)
                else:
                    group_tokens[token1Symbol].aave_withdraw_debt = min(0, group_tokens[token1Symbol].aave_withdraw_debt - token1Amt)
                
                group_tokens[token1Symbol].wallet_debt += token1Amt
                action = 'withdrawn from'
            if trxType in ('Repay', 'RepayBorrow'):
                if protocol == 'Compound':
                    group_tokens[token1Symbol].compound_withdraw_debt = min(0, group_tokens[token1Symbol].compound_withdraw_debt \
                        + token1Amt)
                else:
                    group_tokens[token1Symbol].aave_withdraw_debt = min(0, group_tokens[token1Symbol].compound_withdraw_debt \
                        + token1Amt)

                group_tokens[token1Symbol].wallet_debt = max(0, group_tokens[token1Symbol].wallet_debt - token1Amt)
                action = 'repaid to'
            
            msg_params = {
                'token1Amt': abs(round(token1Amt,2)),
                'token1Symbol': token1Symbol,
                'action': action,
                'funds_location': protocol,
            }

            msg_text = """\n{token1Amt} {token1Symbol} {action} {funds_location}. """.format(**msg_params)
        
        if phase == '1a':
            print(msg_text)
        
        if lockTransac:
            sqlArr.append({
                'trxId': trxId,
                'groupID': x, 
                'blockTime': blockTime,
                'token': token1Symbol, 
                'debtAmtUsd': debtAmtUsd, 
                'freeAmtUsd': freeAmtUsd, 
                'trxType': trxType,
                'protocol': protocol})

        for token, data in group_tokens.items(): 
            msg_params = {
                'token': token,

                'wallet_debt': round(data.wallet_debt, 2),
                'wallet_free': round(data.wallet_free, 2),

                'maker_debt_collat': round(data.maker_debt_collat, 2),
                'maker_free_collat': round(data.maker_free_collat, 2),
                'maker_withdraw_debt': round(data.maker_withdraw_debt, 2),
                
                'compound_debt_collat': round(data.compound_debt_collat, 2),
                'compound_free_collat': round(data.compound_free_collat, 2),
                'compound_withdraw_debt': round(data.compound_withdraw_debt, 2),
                
                'aave_debt_collat': round(data.aave_debt_collat, 2),
                'aave_free_collat': round(data.aave_free_collat, 2),
                'aave_withdraw_debt': round(data.aave_withdraw_debt, 2)
            }

            msg_text = """\n{token}
Wallet balance
    Debt:                           {wallet_debt}
    Free:                           {wallet_free}
Maker balance
    Debt collateral:                {maker_debt_collat}
    Free collateral:                {maker_free_collat}
    Withdrawn debt:                 {maker_withdraw_debt}
Compound balance
    Debt collateral:                {compound_debt_collat}
    Free collateral:                {compound_free_collat}
    Withdrawn debt:                 {compound_withdraw_debt}
Aave balance
    Debt collateral:                {aave_debt_collat}
    Free collateral:                {aave_free_collat}
    Withdrawn debt:                 {aave_withdraw_debt}""".format(**msg_params)
            if phase == '1a':
                print(msg_text)

    if (i % step == 0 or i == (max_id - min_id)) and (phase == '2'):
        ins = algoResults.insert()
        dbConnection.execute(ins, sqlArr)
        sqlArr = []

sum_debt_lock_usd = 0
sum_free_lock_usd = 0

for token, data in sum_tokens.items():
    sum_debt_lock_usd += data.sum_maker_debt_lock_usd + data.sum_compound_debt_lock_usd + data.sum_aave_debt_lock_usd
    sum_free_lock_usd += data.sum_maker_free_lock_usd + data.sum_compound_free_lock_usd + data.sum_aave_free_lock_usd

    if (data.sum_maker_free_lock + data.sum_maker_debt_lock) == 0:
        pct_maker_debt_total = 0
    else: 
        pct_maker_debt_total = round((data.sum_maker_debt_lock / (data.sum_maker_free_lock + data.sum_maker_debt_lock)) * 100, 1)

    if (data.sum_compound_free_lock + data.sum_compound_debt_lock) == 0:
        pct_compound_debt_total = 0
    else: 
        pct_compound_debt_total = round((data.sum_compound_debt_lock / (data.sum_compound_free_lock + data.sum_compound_debt_lock)) * 100, 1)

    if (data.sum_aave_free_lock + data.sum_aave_debt_lock) == 0:
        pct_aave_debt_total = 0
    else: 
        pct_aave_debt_total = round((data.sum_aave_debt_lock / (data.sum_aave_free_lock + data.sum_aave_debt_lock)) * 100, 1)

    msg_params = {
        'token': token,

        'sum_maker_debt_lock': round(data.sum_maker_debt_lock, 2),
        'sum_maker_free_lock': round(data.sum_maker_free_lock, 2),
        'sum_maker_debt_lock_usd': round(data.sum_maker_debt_lock_usd, 2),
        'sum_maker_free_lock_usd': round(data.sum_maker_free_lock_usd, 2),
        'pct_maker_debt_total': pct_maker_debt_total,
        
        'sum_compound_debt_lock': round(data.sum_compound_debt_lock, 2),
        'sum_compound_free_lock': round(data.sum_compound_free_lock, 2),
        'sum_compound_debt_lock_usd': round(data.sum_compound_debt_lock_usd, 2),
        'sum_compound_free_lock_usd': round(data.sum_compound_free_lock_usd, 2),
        'pct_compound_debt_total': pct_compound_debt_total,
        
        'sum_aave_debt_lock': round(data.sum_aave_debt_lock, 2),
        'sum_aave_free_lock': round(data.sum_aave_free_lock, 2),
        'sum_aave_debt_lock_usd': round(data.sum_aave_debt_lock_usd, 2),
        'sum_aave_free_lock_usd': round(data.sum_aave_free_lock_usd, 2),
        'pct_aave_debt_total': pct_aave_debt_total,
    }

    msg_text = """\n{token}
Maker
    Cumulative debt collateral:         {sum_maker_debt_lock:,}
    Cumulative free collateral:         {sum_maker_free_lock:,}
    Cumulative debt collateral (USD):   ${sum_maker_debt_lock_usd:,}
    Cumulative free collateral (USD):   ${sum_maker_free_lock_usd:,}
    Debt percentage of collateral:      {pct_maker_debt_total}%
Compound
    Cumulative debt collateral:         {sum_compound_debt_lock:,}
    Cumulative free collateral:         {sum_compound_free_lock:,}
    Cumulative debt collateral (USD):   ${sum_compound_debt_lock_usd:,}
    Cumulative free collateral (USD):   ${sum_compound_free_lock_usd:,}
    Debt percentage of collateral:      {pct_compound_debt_total}%
Aave
    Cumulative debt collateral:         {sum_aave_debt_lock:,}
    Cumulative free collateral:         {sum_aave_free_lock:,}
    Cumulative debt collateral (USD):   ${sum_aave_debt_lock_usd:,}
    Cumulative free collateral (USD):   ${sum_aave_free_lock_usd:,}
    Debt percentage of collateral:      {pct_aave_debt_total}%""".format(**msg_params)
    
    if phase in ['1a', '1b']:
        print(msg_text)

debt_pct = (sum_debt_lock_usd / (sum_debt_lock_usd + sum_free_lock_usd)) * 100
msg_params = {
    'sum_debt_lock_usd': round(sum_debt_lock_usd, 2),
    'sum_free_lock_usd': round(sum_free_lock_usd, 2),
    'debt_pct': round(debt_pct, 2),
}

msg_text = """\nCumulative debt collateral (USD):   ${sum_debt_lock_usd:,}
Cumulative free collateral (USD):   ${sum_free_lock_usd:,}
Debt percentage of collateral: {debt_pct}%""".format(**msg_params)
if phase == ['1b', '1c']:
    print(msg_text)

100%|██████████| 1/1 [00:00<00:00,  2.13it/s]CPU times: user 453 ms, sys: 0 ns, total: 453 ms
Wall time: 589 ms

